# Transport Calculations (attempt #1)
*Akira Di Sandro, 6/22/20*
<br> In this notebook, I will be making some pseudocode for transport calculations.

## 1. Import Packages as usual

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import sectionate

## 2. Open Dataset

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')

* (6/30) Oday is not available for 'uo' and 'vo'
* Also, I don't think I need 'wo' as a variable

* I think for the line above, I will need to find and download the CM4 dataset that I've been using but with a mask variable (look for it using variable) in order to do the transport calculations.
* (update from 6/30) for the mask, Raf said that I can just use NaN's as an indication that there is no data because of land so just using data from non-NaN suffices

In [5]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [6]:
grid = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
grid

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * bnds       (bnds) float64 1.0 2.0
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    parent_mip_era:          CMIP6
    product:                 model-output
    physics_index:           1
    grid_label:              gn
    parent_source_id:        GFDL-CM4
    mip_era:                 CMIP6
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    institution_id:          NOAA-GFDL
    nominal_resolution:      25 km
    history:                 File was processed by fremetar (GFDL analog of C...
    branch_time_in_child:    0.0
    realm:                   ocean
    references:              see further_info_url attribute
    source_type:             AOGCM
    activity_id:             CMIP
    parent_experiment_id:    piControl
    parent_activity_id:      CMIP
    frequency:               mon
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    table_id:                Omon
    contact:                 gfdl.climate.model.info@noaa.gov
    realization_index:       1
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    source_id:               GFDL-CM4
    experiment_id:           historical
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    status:                  2019-08-06;created;by nhn2@columbia.edu
    branch_method:           standard
    parent_time_units:       days since 0001-1-1
    comment:                 <null ref>
    forcing_index:           1
    experiment:              historical
    parent_variant_label:    r1i1p1f1
    sub_experiment:          none
    variant_label:           r1i1p1f1
    branch_time_in_parent:   36500.0
    initialization_index:    1
    data_specs_version:      01.00.27
    sub_experiment_id:       none
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    variant_info:            N/A
    institution:             National Oceanic and Atmospheric Administration,...
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

* The above two lines will also be dependent on the change in line 4.

In [7]:
dog1 = col.search(source_id='GFDL-CM4', variable_id='volcello', grid_label='gn')
dog1.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-CM4,1pctCO2,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/1pctCO2/r1i...,NaN,20180701
1,CMIP,NOAA-GFDL,GFDL-CM4,abrupt-4xCO2,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/abrupt-4xCO...,NaN,20180701
2,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...,NaN,20180701
3,CMIP,NOAA-GFDL,GFDL-CM4,piControl,r1i1p1f1,Omon,volcello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r...,NaN,20180701
4,ScenarioMIP,NOAA-GFDL,GFDL-CM4,ssp245,r1i1p1f1,Omon,volcello,gn,gs://cmip6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp2...,NaN,20180701
5,ScenarioMIP,NOAA-GFDL,GFDL-CM4,ssp585,r1i1p1f1,Omon,volcello,gn,gs://cmip6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp5...,NaN,20180701


In [8]:
dset_dict1 = dog1.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict1.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.abrupt-4xCO2.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.1pctCO2.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn',
 'ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp245.Omon.gn',
 'CMIP.NOAA-GFDL.GFDL-CM4.piControl.Omon.gn',
 'ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.Omon.gn']

In [9]:
grid1 = dset_dict1['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [11]:
grid1

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, vertex: 4, x: 1440, y: 1080)
Coordinates:
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * bnds       (bnds) float64 1.0 2.0
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    volcello   (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 2, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   36500.0
    comment:                 <null ref>
    contact:                 gfdl.climate.model.info@noaa.gov
    creation_date:           2019-02-24T05:33:48Z
    data_specs_version:      01.00.27
    experiment:              historical
    experiment_id:           historical
    external_variables:      areacello volcello
    forcing_index:           1
    frequency:               mon
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    grid_label:              gn
    history:                 File was processed by fremetar (GFDL analog of C...
    initialization_index:    1
    institution:             National Oceanic and Atmospheric Administration,...
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    mip_era:                 CMIP6
    nominal_resolution:      25 km
    parent_activity_id:      CMIP
    parent_experiment_id:    piControl
    parent_mip_era:          CMIP6
    parent_source_id:        GFDL-CM4
    parent_time_units:       days since 0001-1-1
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   ocean
    references:              see further_info_url attribute
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    source_id:               GFDL-CM4
    source_type:             AOGCM
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Omon
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    tracking_id:             hdl:21.14100/79122291-5823-4e21-8e3e-0f59a9ff542...
    variable_id:             volcello
    variant_info:            N/A
    variant_label:           r1i1p1f1
    status:                  2019-10-14;created;by nhn2@columbia.edu
    intake_esm_varname:      volcello
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

## 3. Open Cluster 

In [10]:
from dask.distributed import Client
from dask_gateway import Gateway


gateway = Gateway()  # connect to Gateway

cluster = gateway.new_cluster()  # create cluster
cluster.scale(10)  # scale cluster

client = Client(cluster)  # connect Client to Cluster

In [11]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.8226ae8642614d9c8c7f6711f0c4ab33 Dashboard: https://us-central1-b.gcp.pangeo.io/services/dask-gateway/clusters/prod.8226ae8642614d9c8c7f6711f0c4ab33/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## 4. Actual Calculations

### a. Calculations to get the U/V points

* (6/30) still need to edit these lat and lon to match the observational data

In [12]:
lat1 = -4.49531
lon1 = -208.21236
lat2 = -5.989064
lon2 = -205.234668

In [14]:
isec, jsec, xsec, ysec = sectionate.create_section(grid['lon'], grid['lat'], lon1, lat1, lon2, lat2)

best fit is rounding down


In [17]:
uvpoints = sectionate.transports_C.MOM6_UVpoints_from_section(isec, jsec)
udata = np.empty((6,4))
vdata = np.empty((11,4))
ucount = 0 
vcount = 0

for i in range(len(uvpoints)):
    point = uvpoints[i]
    pttype, i, j = point
    i = int(i)
    j = int(j)
    if pttype == 'U':
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        udata[ucount] = [i, j, lon, lat]  
        ucount += 1
    else:
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        vdata[vcount] = [i, j, lon, lat]  
        vcount += 1
# print(f'{point[0]}, {point[1]}, {point[2]}, {lon}, {lat}')

In [1]:
udata

NameError: name 'udata' is not defined

udata should be a 4-by-uvpoints that are u, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the u point.
<br>udata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]
         
vdata should be a 4-by-uvpoints that are v, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the v point.
<br>vdata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]

### b. Using U/V points to calculate transport
The following are the steps I cam eup with to do the calculation. <br>
* (transport for each cell) T = u * SA (of cell face) * mask
* SA = dy (vertical distance of cell) * depth of cell     (6/30) SA can be replaced with areacello
* Mask = fraction that tells us how much of the cell is in water (mostly 1 but for the edges, it will be some fraction<1)
* Do this calculation for all depths of (i,j) cell and summate


In [19]:
# masked area code from Raf's template
#    this takes care of multiplying the SA by the mask, so all we have to do is use this area 
#    data for transport calculations
area_masked = grid1['areacello'].where(~np.isnan(grid['uo'].isel(time=0)))

In [57]:
grid1

<xarray.Dataset>
Dimensions:    (member_id: 1, vertex: 4, x: 1440, y: 1080)
Coordinates:
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    areacello  (member_id, y, x) float32 dask.array<chunksize=(1, 1080, 1440), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    0.0
    branch_time_in_parent:   54750.0
    comment:                 <null ref>
    contact:                 gfdl.climate.model.info@noaa.gov
    creation_date:           2018-11-20T18:41:34Z
    data_specs_version:      01.00.27
    experiment:              pre-industrial control
    experiment_id:           piControl
    forcing_index:           1
    frequency:               fx
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    grid_label:              gn
    history:                 File was processed by fremetar (GFDL analog of C...
    initialization_index:    1
    institution:             National Oceanic and Atmospheric Administration,...
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    mip_era:                 CMIP6
    nominal_resolution:      25 km
    parent_activity_id:      CMIP
    parent_experiment_id:    piControl-spinup
    parent_mip_era:          CMIP6
    parent_source_id:        GFDL-CM4
    parent_time_units:       days since 1700-1-1
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   ocean
    references:              see further_info_url attribute
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    source_id:               GFDL-CM4
    source_type:             AOGCM
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Ofx
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    tracking_id:             hdl:21.14100/82865d31-bc4e-437e-93d7-f37fd704ca65
    variable_id:             areacello
    variant_info:            N/A
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    intake_esm_varname:      areacello
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn

In [48]:
area_masked

<xarray.DataArray 'areacello' (member_id: 1, y: 1080, x: 1440, lev: 35)>
dask.array<where, shape=(1, 1080, 1440, 35), dtype=float32, chunksize=(1, 1080, 1440, 35), chunktype=numpy.ndarray>
Coordinates:
    lon        (y, x) float32 -299.71835 -299.47037 ... 59.998463 59.99949
    lat        (y, x) float32 -79.80674 -79.80527 ... 64.22252 64.11359
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
    time       object 1850-01-16 12:00:00
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
Attributes:
    cell_methods:   area: sum
    long_name:      Grid-Cell Area
    original_name:  areacello
    standard_name:  cell_area
    units:          m2

* The problem with the code below is that u is read-only. I'm going to try to just cpy each element of grid['uo'] into the array u instead.

In [49]:
total = []  # array that will store transport across each point in udata
for pt in udata:
    i, j, lon, lat = pt
    uo = grid['uo'].isel(x=int(i),y=int(j),time=0).values[0]
    u = []
    for i in range(len(uo)):
        if np.isnan(uo[i]):
            u = np.append(u, 0)
        else:
            u = np.append(u, uo[i])
    SAm = area_masked.isel(x=int(i),y=int(j)).values[0]  # masked surface area of cell
    SA = []
    for i in range(len(SAm)):
        if np.isnan(SAm[i]):
            SA = np.append(SA, 0)
        else:
            SA = np.append(SA, SAm[i])
    pt_total = 0
    for row in range(len(SA)):   # row iterates through the different depths that aren't nan
        pt_total += u[row] * SA[row]
        print(row,u[row],SA[row],pt_total, pt_total/10**6)
    total = np.append(total, pt_total)
utrans = np.sum(total)
utrans

0 0.0 769643520.0 0.0 0.0
1 0.0 769643520.0 0.0 0.0
2 0.0 769643520.0 0.0 0.0
3 0.0 769643520.0 0.0 0.0
4 0.0 769643520.0 0.0 0.0
5 0.0 769643520.0 0.0 0.0
6 0.0 769643520.0 0.0 0.0
7 0.0 769643520.0 0.0 0.0
8 0.0 769643520.0 0.0 0.0
9 0.0 769643520.0 0.0 0.0
10 0.0 769643520.0 0.0 0.0
11 0.0 769643520.0 0.0 0.0
12 0.0 769643520.0 0.0 0.0
13 0.0 769643520.0 0.0 0.0
14 0.0 769643520.0 0.0 0.0
15 0.0 769643520.0 0.0 0.0
16 0.0 769643520.0 0.0 0.0
17 0.0 769643520.0 0.0 0.0
18 0.0 769643520.0 0.0 0.0
19 0.0 769643520.0 0.0 0.0
20 0.0 769643520.0 0.0 0.0
21 0.0 769643520.0 0.0 0.0
22 0.0 769643520.0 0.0 0.0
23 0.0 769643520.0 0.0 0.0
24 0.0 769643520.0 0.0 0.0
25 0.0 769643520.0 0.0 0.0
26 0.0 769643520.0 0.0 0.0
27 0.0 769643520.0 0.0 0.0
28 0.0 769643520.0 0.0 0.0
29 0.0 0.0 0.0 0.0
30 0.0 0.0 0.0 0.0
31 0.0 0.0 0.0 0.0
32 0.0 0.0 0.0 0.0
33 0.0 0.0 0.0 0.0
34 0.0 0.0 0.0 0.0
0 0.07202210277318954 769099072.0 55392132.406348705 55.39213240634871
1 0.05362412706017494 769099072.0 96634398

2645769293.6392655

Now, run the same calculation for 'vo'.

In [44]:
total = []  # array that will store transport across each point in vdata
for pt in vdata:
    i, j, lon, lat = pt
    vo = grid['vo'].isel(x=int(i),y=int(j),time=0).values[0]
    v = []
    for i in range(len(vo)):
        if np.isnan(vo[i]):
            v = np.append(v, 0)
        else:
            v = np.append(v, vo[i])
    SAm = area_masked.isel(x=int(i),y=int(j)).values[0]  # masked surface area of cell
    SA = []
    for i in range(len(SAm)):
        if np.isnan(SAm[i]):
            SA = np.append(SA, 0)
        else:
            SA = np.append(SA, SAm[i])
    pt_total = 0
    for row in range(len(SA)):   # row iterates through the different depths that aren't nan
        pt_total += v[row] * SA[row]
    total = np.append(total, pt_total)
vtrans = np.sum(total)

Now, add utrans and vtrans to get the total transport across the cross-section.

In [48]:
transport = (utrans + vtrans) * (10**-6)
transport

-6779.613910196594

The units look a little wonky, but this might be due to the fact that the time stamp is very early. That being said, some part of my calculation may be flawed.

### c. Using the above code to make a function that calculates the transport for each time stamp

In [21]:
def calc_model_transport(ds, t_step, udata, vdata, mask):
    total = []
    for pt in udata:
        i,j,lon,lat = pt
        uo = ds['uo'].isel(x=int(i),y=int(j),time=t_step).values[0]  #uo (m/s)
        u = []                                                       #u (m/s)
        for i in range(len(uo)):
            if np.isnan(uo[i]):
                u = np.append(u, 0)
            else:
                u = np.append(u, uo[i])
        SAm = mask.isel(x=int(i),y=int(j)).values[0]
        SA = []                                                      #SA (m^2)             
        for i in range(len(SAm)):
            if np.isnan(SAm[i]):
                SA = np.append(SA, 0)
            else:
                SA = np.append(SA, SAm[i])
        pt_total=0
        for row in range(len(SA)):
            pt_total += u[row] * SA[row]                        #pt_total (m^3/s)
        total = np.append(total, pt_total)
    utrans = np.sum(total)                                      #utrans (m^3/s)
    total = []
    for pt in vdata:
        i,j,lon,lat = pt
        vo = ds['vo'].isel(x=int(i),y=int(j),time=t_step).values[0]
        v = []
        for i in range(len(vo)):
            if np.isnan(vo[i]):
                v = np.append(v, 0)
            else:
                v = np.append(v, vo[i])
        SAm = mask.isel(x=int(i),y=int(j)).values[0]
        SA = []
        for i in range(len(SAm)):
            if np.isnan(SAm[i]):
                SA = np.append(SA, 0)
            else:
                SA = np.append(SA, SAm[i])
        pt_total=0
        for row in range(len(SA)):
            pt_total += v[row] * SA[row]
        total = np.append(total, pt_total)
    vtrans = np.sum(total)
    transport = (utrans + vtrans) * (10**-6)          #converting (m^3/s) to (Sv)
    return transport

In [22]:
trans_time0 = calc_model_transport(grid,0,udata,vdata,area_masked)
trans_time0

-6779.613910196594

## 5. Finding the times to plug into the transport calculations

In [40]:
grid['time'][1950]

<xarray.DataArray 'time' ()>
array(cftime.DatetimeNoLeap(2012-07-16 12:00:00), dtype=object)
Coordinates:
    time     object 2012-07-16 12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  noleap
    description:    Temporal mean
    long_name:      time
    standard_name:  time

In [41]:
trans_time0 = calc_model_transport(grid,1950,udata,vdata,area_masked)
trans_time0

9491.395869182112

## Closing Clusters after use

In [62]:
client.close()
cluster.close()